In [1]:
import cogent3
from cogent3 import get_app
from phylim.apps import phylim
import libs
import paths

In [2]:
folder_in = paths.DATA_APES114 + 'intronsAR/'
in_dstore = cogent3.open_data_store(folder_in, suffix='fa', mode='r')

print(in_dstore.describe)

Directory datastore
record type      number
-----------------------
completed            25
not_completed         0
logs                  0
-----------------------


In [3]:
#perform a codon alignment and remove stop codons
loader = get_app("load_aligned", moltype="dna")
omit_degs = get_app("omit_degenerates", moltype="dna")
rename = libs.renamer_aligned()
concat = get_app("concat", moltype="dna")

app = loader + omit_degs + rename
#By using this renamer_aligned function I throw away sequences with paralogs
results_allpos = [r for r in app.as_completed(in_dstore[:], parallel=True) if r]
alpos_alns = concat(results_allpos)
alpos_alns.source = "intronsAR_alignments"
alpos_alns

   0%|          |00:00<?

,0
Chimpanzee,TGTGCTCATTATTTCACTAATCTTTTCTTGTGTAATCCATGAAGATCTTGCAGGCTTGAT
Human,............................................................
Gorilla,............................................................


In [4]:
#Get distances of concatenated sequences 
#time_het=max sets a 36 params substitution model
sm = get_app("model", "GN", time_het="max", optimise_motif_probs=True, show_progress=True)
result_intergenic = sm(alpos_alns)

checker = get_app("phylim")
checked = checker(result_intergenic)
lengthstree = result_intergenic.lf.get_lengths_as_ens()
print("Is model identifiable? " + str(checked.is_identifiable) + "\n")
print("Model results : \n" + str(result_intergenic.lf) + "\n")
print("Distances: " + str(lengthstree))

   0%|          |00:00<?

   0%|          |00:00<?

Is model identifiable? True

Model results : 
GN
log-likelihood = -157178.2749
number of free parameters = 39
edge          parent    length     A>C     A>G     A>T     C>A     C>G
----------------------------------------------------------------------
Chimpanzee    root        0.01    1.14    3.41    0.89    1.38    1.50
Human         root        0.01    1.68    6.54    0.72    1.99    2.77
Gorilla       root        0.01    0.64    3.29    0.58    1.03    1.08
----------------------------------------------------------------------

continued: 
  C>T      G>A     G>C     G>T     T>A     T>C
----------------------------------------------
 5.56     5.04    1.12    1.56    1.30    3.28
14.98    13.93    3.07    2.09    1.85    6.62
 4.79     4.97    1.14    0.67    0.65    3.37
----------------------------------------------

   A       C       G       T
----------------------------
0.24    0.26    0.25    0.25
----------------------------

Distances: {'Chimpanzee': np.float64(0.011914876774